# Imports

In [1]:
import transformers
import torch
import random
import numpy as np
from torch.utils.data import random_split
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import GenerationConfig

import sys
if '..' not in sys.path: sys.path.append('..')
from src.data.make_dataset import load_detoxification_dataset, load_toxicity_dataset

# Load the pretrained T5

In [2]:
global_seed = 1984

transformers.set_seed(global_seed)
random.seed(global_seed)
np.random.seed(global_seed)
torch.manual_seed(global_seed)
torch.cuda.manual_seed_all(global_seed)
model_checkpoint = "t5-small"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
dataset_portion = 1
dataset_kwargs = {
    'path': '../data/raw/filtered.tsv', # path to raw data
    'cache_path': '../data/interim/tokenized.tsv', # path to processed data (or where to store it)
    'tokenizer': tokenizer, # tokenizer to tokenize texts
    'portion': dataset_portion # get only a portion of dataset [0..1]
}

# Dataset

In [4]:
dataset = load_detoxification_dataset(**dataset_kwargs)

val_ratio = 0.2
train_dataset, val_dataset = random_split(dataset, [1 - val_ratio, val_ratio])

# Training

In [5]:
# defining the parameters for training
genConfig = GenerationConfig.from_pretrained(model_checkpoint)
genConfig.max_new_tokens = 64

batch_size = 32
postfix = "-10"
save_model_path = f'../models/t5_detoxifier{postfix}x10lr'
args = Seq2SeqTrainingArguments(
    f"../models/{model_checkpoint}-detoxification{postfix}x10lr",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
    logging_steps=5000,
    save_steps=25000,
    generation_config=genConfig
)

In [6]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [7]:
trainer.train()

  0%|          | 0/288900 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.96, 'learning_rate': 0.00049134994807892, 'epoch': 0.35}
{'loss': 1.8517, 'learning_rate': 0.0004826998961578401, 'epoch': 0.69}


  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 1.6692404747009277, 'eval_runtime': 112.3317, 'eval_samples_per_second': 1028.694, 'eval_steps_per_second': 32.155, 'epoch': 1.0}
{'loss': 1.8016, 'learning_rate': 0.0004740515749394254, 'epoch': 1.04}
{'loss': 1.7309, 'learning_rate': 0.0004654015230183455, 'epoch': 1.38}
{'loss': 1.7211, 'learning_rate': 0.0004567532017999308, 'epoch': 1.73}


...

  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 1.518808364868164, 'eval_runtime': 92.0118, 'eval_samples_per_second': 1255.872, 'eval_steps_per_second': 39.256, 'epoch': 19.0}
{'loss': 1.1836, 'learning_rate': 2.4257528556593975e-05, 'epoch': 19.04}
{'loss': 1.1723, 'learning_rate': 1.5607476635514018e-05, 'epoch': 19.38}
{'loss': 1.1693, 'learning_rate': 6.95742471443406e-06, 'epoch': 19.73}


  0%|          | 0/3612 [00:00<?, ?it/s]

{'eval_loss': 1.5204483270645142, 'eval_runtime': 91.1479, 'eval_samples_per_second': 1267.775, 'eval_steps_per_second': 39.628, 'epoch': 20.0}
{'train_runtime': 33602.6877, 'train_samples_per_second': 275.11, 'train_steps_per_second': 8.598, 'train_loss': 1.406808580904941, 'epoch': 20.0}


TrainOutput(global_step=288900, training_loss=1.406808580904941, metrics={'train_runtime': 33602.6877, 'train_samples_per_second': 275.11, 'train_steps_per_second': 8.598, 'train_loss': 1.406808580904941, 'epoch': 20.0})

In [8]:
# saving model
trainer.save_model(save_model_path)

In [64]:
# loading the model and run inference for it
# model = AutoModelForSeq2SeqLM.from_pretrained(save_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained('../models backup/t5_detoxifier-10')
model.eval()
model.config.use_cache = False

# Testing ??

In [65]:
def translate(model, inference_request, tokenizer=tokenizer):
    tokenized = tokenizer.encode(inference_request, return_tensors="pt")
    outputs = model.generate(tokenized, generation_config=genConfig)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [76]:
inference_request = """
this guy is a con man. He's an actor. He's just a character.
"""
translate(model, inference_request)

This guy is a con, he's an actor, he's just a character.


# Validation ????

In [13]:
from src.models.t5_toxicity_evaluator import T5TEModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
evalutator = T5TEModel('../models/last_toxic_regressor/model.pt').to(device)
model.to(device)
_ = evalutator.model.eval()

In [14]:
eval_dataset = load_toxicity_dataset(**dataset_kwargs)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=128, shuffle=False, collate_fn=evalutator.collate_batch)

In [15]:
from tqdm.auto import tqdm
transformed = []

for batch in tqdm(eval_loader, total=len(eval_loader), desc='Translating'):
    output = model.generate(input_ids=batch.input_ids, attention_mask=batch.attention_mask, generation_config=genConfig)
    transformed += output.detach().cpu()

Translating:   0%|          | 0/452 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [16]:
ref_evaluations = []

torch.cuda.empty_cache()
for batch in tqdm(eval_loader, total=len(eval_loader), desc='Evaluation'):
    output = evalutator(batch)
    ref_evaluations += output.detach().cpu()

Evaluation:   0%|          | 0/452 [00:00<?, ?it/s]

In [17]:
transformed_keys = [{'input_ids': x} for x in transformed]
trn_loader = torch.utils.data.DataLoader(transformed_keys, batch_size=128, shuffle=False, collate_fn=evalutator.collate_batch)

In [18]:
trn_evaluations = []

torch.cuda.empty_cache()
for batch in tqdm(trn_loader, total=len(trn_loader), desc='Evaluation'):
    output = evalutator(batch)
    trn_evaluations += output.detach().cpu()

Evaluation:   0%|          | 0/452 [00:00<?, ?it/s]

In [19]:
torch.cuda.empty_cache()

In [20]:
threshold = 0.5

refevs = np.array(ref_evaluations)
trnevs = np.array(trn_evaluations)

ref_toxs = refevs > threshold
trn_toxs = trnevs > threshold

In [21]:
ref_neutrals = ref_toxs == False
ref_toxics = ref_toxs == True
trn_neutrals = trn_toxs == False
trn_toxics = trn_toxs == True

print(f'Neutral -> neutral: {np.sum(ref_neutrals)} -> {np.sum(np.logical_and(ref_neutrals, trn_neutrals))}')
print(f'Neutral -> toxic: {np.sum(ref_neutrals)} -> {np.sum(np.logical_and(ref_neutrals, trn_toxics))}')
print(f'Toxic -> neutral: {np.sum(ref_toxics)} -> {np.sum(np.logical_and(ref_toxics, trn_neutrals))}')
print(f'Toxic -> toxic: {np.sum(ref_toxics)} -> {np.sum(np.logical_and(ref_toxics, trn_toxics))}')

Neutral -> neutral: 27871 -> 27857
Neutral -> toxic: 27871 -> 14
Toxic -> neutral: 29906 -> 28996
Toxic -> toxic: 29906 -> 910
